In [23]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

np.random.seed(42)

In [24]:
class BaseOptimizer:
    def __init__(self, num_func_calls = 0, num_grad_calls = 1):
        self.num_func_calls = num_func_calls
        self.num_grad_calls = num_grad_calls
    
    def step(self, func, grad, x, *args):
        raise NotImplementedError()
    
class ConstantOptimizer(BaseOptimizer):
    def __init__(self, learning_rate=0.01):
        super().__init__()
        self.learning_rate = learning_rate

    def step(self, func, grad, x, *args):
        return x - self.learning_rate * grad(x)

class PolynomialOptimizer(ConstantOptimizer):
    def __init__(self, alpha=0.5, beta=1, **kwargs):
        super().__init__(**kwargs)
        self.iter = 0
        self.beta = beta
        self.alpha = alpha

    def step(self, func, grad, x):
        self.iter += 1
        return x - self.learning_rate / (1 + self.beta * self.iter) ** self.alpha * grad(x)

class AdaptiveOptimizer(ConstantOptimizer):
    def step(self, func, grad, x):
        gradient = grad(x)
        return x - self.learning_rate * gradient / np.linalg.norm(gradient)

class GradientDescent:
    def __init__(self, max_iter=1000, tolerance=1e-4):
        self.max_iter = max_iter
        self.tolerance = tolerance
        self.num_func_calls = None
        self.num_grad_calls = None
        self.num_iter = None

    def optimize(self, func, grad, start_point, optimizer=ConstantOptimizer()):
        self.num_func_calls = 0
        self.num_grad_calls = 0
        self.num_iter = 0
        x = np.array(start_point, dtype=float)
        history = [x.copy()]

        for iter in range(self.max_iter):
            x = optimizer.step(func, grad, x)
            self.num_func_calls += optimizer.num_func_calls
            self.num_grad_calls += optimizer.num_grad_calls
            history.append(x.copy())

            if np.linalg.norm(grad(x)) < self.tolerance:
                self.num_iter = iter
                break
        self.num_iter = self.max_iter
        return x, np.array(history)


In [25]:
class FunctionFactory:
    class QuadraticFunction:
        def __init__(self, a = 0, b = 0, c = 0, d = 0, e = 0, f = 0):
            self.a = a
            self.b = b
            self.c = c
            self.d = d
            self.e = e
            self.f = f

        def __call__(self, x):
            return self.a * x[0] ** 2 + self.b * x[1] ** 2 + self.c * x[0] * x[1] + self.d * x[0] + self.e * x[1] + self.f
        
        def grad(self, x):
            return np.array([2 * self.a * x[0] + self.c * x[1] + self.d, 2 * self.b * x[1] + self.c * x[0] + self.e])
        
        def find_extremum(self):
            A = np.array([[2 * self.a, self.c],
                          [self.c, 2 * self.b]])
            B = np.array([-self.d, -self.e])

            return np.linalg.solve(A, B)

    def get_from_coefficients(self, a = 0, b = 0, c = 0, d = 0, e = 0, f = 0):
        return self.QuadraticFunction(a, b, c, d, e, f)

    def get_random(self):
        a, b = np.random.uniform(0.1, 1.0, 2)
        max_c = 2 * np.sqrt(a * b) * 0.9
        c = np.random.uniform(-max_c, max_c)
        d, e = np.random.uniform(-1.0, 1.0, 2)
        f = np.random.uniform(-10.0, 10.0)
        return self.QuadraticFunction(a, b, c, d, e, f)

    def get_from_eigenvalues(self, l1=1, l2=1, B=None, C=0, theta=None):
        if theta is None:
            theta = np.random.uniform(0, 2 * np.pi)

        Q = np.array([
            [np.cos(theta), -np.sin(theta)],
            [np.sin(theta), np.cos(theta)]
        ])
        D = np.diag([l1, l2])
        A = Q @ D @ Q.T
        if B is None:
            B = np.random.uniform(-1, 1, size=2)

        return self.QuadraticFunction(A[0, 0], A[1, 1], 2 * A[0, 1], B[0], B[0], C)

In [26]:
def build_chart(f, path, center=(0, 0), rx=5, ry=5):
    try:
        x_star = f.find_extremum()
    except AttributeError:
        x_star = np.array(center)
    values = [f(p) for p in path]
    X, Y = np.meshgrid(np.linspace(x_star[0] - rx, x_star[0] + rx, 100), np.linspace(x_star[1] - ry, x_star[1] + ry, 100))
    Z = f([X, Y])

    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{"type": "surface"}, {"type": "xy"}]],
        column_widths=[0.7, 0.3]
    )
    fig.add_trace(
        go.Surface(x=X, y=Y, z=Z, colorscale='Viridis', opacity=0.7, showscale=False),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter3d(x=path[:, 0], y=path[:, 1], z=values, marker=dict(size=1, color='red')),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=np.arange(len(values)), y=values-f(x_star)),
        row=1, col=2
    )
    fig.show()

In [27]:
# Молчанов В.
class GoldenOptimizer(ConstantOptimizer):
    def __init__(self, tolerance=1e-3, **kwargs):
        super().__init__(**kwargs)
        self.tolerance = tolerance
        self.gr = (np.sqrt(5) - 1) / 2

    def step(self, func, grad, x):
        a, b = x, x - self.learning_rate * grad(x)
        c, d = b - self.gr * (b - a), a + self.gr * (b - a)
        fc, fd = func(c), func(d)
        self.num_func_calls = 2
        while np.linalg.norm(b - a) > self.tolerance:
            if fc < fd:
                b, d = d, c
                fd, c = fc, b - self.gr * (b - a)
                fc = func(c)
            else:
                a, c = c, d
                fc, d = fd, a + self.gr * (b - a)
                fd = func(d)
            self.num_func_calls += 1
        return (a + b) / 2

In [28]:
# Молчанов В.
class ArmijoOptimizer(BaseOptimizer):
    """
    Определение длины следующего шага методом Армихо с использованием backtracking
    """
    def __init__(self, alpha_0=1.0, q=0.5, c1=1e-4):
        super().__init__()
        self.alpha_0 = alpha_0
        self.q = q
        self.c1 = c1

    def step(self, func, grad, x):
        self.num_func_calls = 1
        alpha = self.alpha_0
        gradient = grad(x)
        fx = func(x)
        while func(x - alpha * gradient) > fx - self.c1 * alpha * gradient @ gradient:
            self.num_func_calls += 1
            alpha *= self.q
        return x - alpha * gradient

In [29]:
# Молчанов В.
class WolfeOptimizer(BaseOptimizer):
    """
    Определение длины следующего шага методом Вульфа с использованием бинарного поиска
    """
    def __init__(self, c1=1e-4, c2=0.9, max_iter = 100):
        """
        Требование: c1 < c2
        """
        super().__init__()
        self.c1 = c1
        self.c2 = c2
        self.max_iter = max_iter

    def step(self, func, grad, x):
        self.num_func_calls = 1
        self.num_grad_calls = 1
        gradient = grad(x)
        f_curr = func(x)
        left = 1
        armijo = True
        x_new = x
        for _ in range(self.max_iter):
            left *= 2
            x_new = x - left * gradient
            armijo = func(x_new) <= f_curr - self.c1 * left * gradient @ gradient
            self.num_func_calls += 1
            if not armijo:
                break
        right = 0
        for _ in range(self.max_iter):
            middle = (left + right) / 2
            x_new = x - middle * gradient
            armijo = func(x_new) <= f_curr - self.c1 * middle * gradient @ gradient
            curvature = grad(x_new) @ -gradient >= self.c2 * gradient @ -gradient
            self.num_func_calls += 1
            self.num_grad_calls += 1
            if armijo and curvature:
                break
            elif not armijo:
                left = middle
            else:
                right = middle
        return x_new

In [50]:
factory = FunctionFactory()
func = factory.get_from_eigenvalues(2, 1)
build_chart(func, np.array([[0, 0]]))

In [ ]:
# Зайцев З.

# https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize

from scipy.optimize import minimize_scalar, line_search, minimize

# Определим целевую функцию
def func(x):
    return (x - 2) ** 2

# Градиент функции для использования в line_search
def grad_func(x):
    return 2 * (x - 2)

# 1. Использование метода золотого сечения через minimize_scalar
# Минимизация функции на интервале [0, 4] методом золотого сечения
result_golden = minimize_scalar(func, method='golden', bracket=(0, 4), options={'disp': True})
print(f"\nМинимум функции (метод золотого сечения) при x = {result_golden.x}")
print(f"Число шагов: {result_golden.nit}")  # Количество шагов
print(f"Точность: {result_golden.fun}")  # Точность (значение функции в минимуме)


# https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.line_search.html

# 2. Использование метода Армихо через line_search
# Применим line_search для нахождения шага с методом Армихо
x0 = np.array([3.0])  # начальная точка
search_result_armijo = line_search(func, grad_func, x0, -grad_func(x0), c1=1e-4)
print(f"\nШаг метода Армихо: {search_result_armijo[0]}")
print(f"Количество шагов: {len(search_result_armijo)}")  # Шаги, возвращаемые line_search

# 3. Использование метода Вульфа через line_search
# Применим line_search с критериями Вульфа
search_result_wolfe = line_search(func, grad_func, x0, -grad_func(x0), c1=1e-4, c2=0.9)
print(f"\nШаг метода Вульфа: {search_result_wolfe[0]}")
print(f"Количество шагов: {len(search_result_wolfe)}")  # Шаги, возвращаемые line_search

